In [1]:
from fechas import *
import pandas as pd
from Datasets import   generate_dataset, up_to_db
from binance.spot import Spot
from conexion import abrir_conexion_MYSQL,open_conexion_get

In [3]:
apy_key, secret_key, passw = get_credentials("Data/tesnet.txt")

cliente = Spot(key=apy_key,secret=secret_key)

In [47]:
# Para tesnet
cliente = Spot(key=apy_key,secret=secret_key,base_url="https://testnet.binance.vision")

In [65]:
# Balance
info = cliente.account()
df_balance = pd.DataFrame(info["balances"]).sort_values("free",ascending=False)
df_balance[df_balance["free"].astype(float)>0]

,asset,free,locked
3,ETH,98.00000000,0.00000000
5,TRX,500000.00000000,0.00000000
7,XRP,50000.00000000,0.00000000
4,LTC,500.00000000,0.00000000
6,USDT,10665.88437080,0.00000000
2,BUSD,10000.00000000,0.00000000
0,BNB,1000.00000000,0.00000000
1,BTC,1.10000000,0.00000000


In [41]:
df = generate_dataset(cliente,"ETHUSDT","15m","2023-08-01 00:00:00")
df

,Open_Time,Open,High,Low,Close,Volumne,Close_Time,Quote_asset_vol,Number_trades,Taker_buy_base,Taker_buy_quote,Ignore
0,2023-08-02 05:45:00,1819.42,1819.42,1817.65,1817.65,1.05000,2023-08-02 05:59:59.999,1910.347100,53,1.00000,1819.420000,0
1,2023-08-02 06:00:00,1817.28,1819.42,1439.91,1784.98,4.22995,2023-08-02 06:14:59.999,7665.658959,68,3.29295,5991.259089,0
2,2023-08-02 06:15:00,1784.15,1857.59,1557.55,1857.59,189.77121,2023-08-02 06:29:59.999,346349.952415,591,166.28993,303227.083949,0
3,2023-08-02 06:30:00,1857.58,1857.58,1632.53,1855.80,296.41848,2023-08-02 06:44:59.999,549730.759930,845,134.14961,249084.206963,0
4,2023-08-02 06:45:00,1855.80,1855.80,1845.65,1854.24,269.03607,2023-08-02 06:59:59.999,498938.455709,770,240.81194,446616.149405,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3224,2023-09-04 19:45:00,1626.35,1626.78,1623.01,1623.73,200.36894,2023-09-04 19:59:59.999,325511.984128,552,99.42434,161534.215887,0
3225,2023-09-04 20:00:00,1623.34,1624.58,1622.26,1622.65,203.40158,2023-09-04 20:14:59.999,330235.631196,525,107.33554,174279.943453,0
3226,2023-09-04 20:15:00,1622.63,1623.51,1621.54,1622.40,201.20973,2023-09-04 20:29:59.999,326474.477312,519,105.07284,170499.861841,0
3227,2023-09-04 20:30:00,1622.14,1623.41,1614.44,1619.70,265.02456,2023-09-04 20:44:59.999,429523.311006,785,144.50671,234209.037441,0


In [38]:
par_de_trading = "ETHUSDT"

In [35]:
# Post a new order
params = {
    # Necesary by LIMIT, MARKET
    'symbol': 'ETHUSDT',
    'side': 'SELL',  # [SELL, BUY]
    'type': 'MARKET', # [MARKET, LIMIT, STOP_LIMIT, STOP_MARKET]
    'quantity': 1,  # Porcentaje de side
    # Necesary by LIMIT
    #"timeInForce": "GTC",
    #"price":1620
}

response = cliente.new_order(**params)
print(response)

{'symbol': 'ETHUSDT', 'orderId': 12961820, 'orderListId': -1, 'clientOrderId': 'h1yRjMxd5q8PCEH1gq3hxv', 'transactTime': 1693881755642, 'price': '0.00000000', 'origQty': '1.00000000', 'executedQty': '1.00000000', 'cummulativeQuoteQty': '1618.95343000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'workingTime': 1693881755642, 'fills': [{'price': '1618.96000000', 'qty': '0.58680000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'tradeId': 1895165}, {'price': '1618.95000000', 'qty': '0.36444000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'tradeId': 1895166}, {'price': '1618.90000000', 'qty': '0.04876000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'tradeId': 1895167}], 'selfTradePreventionMode': 'NONE'}


In [40]:
# Trades que he hecho en cierto simbolo
my_trades = cliente.my_trades(symbol=par_de_trading,)
pd.DataFrame(my_trades)

,symbol,id,orderId,orderListId,price,qty,quoteQty,commission,commissionAsset,time,isBuyer,isMaker,isBestMatch
0,ETHUSDT,1894943,12960909,-1,1620.36000000,0.36412000,590.00548320,0.00000000,USDT,1693881564753,False,False,True
1,ETHUSDT,1894944,12960909,-1,1620.34000000,0.56779000,920.01284860,0.00000000,USDT,1693881564753,False,False,True
2,ETHUSDT,1894945,12960909,-1,1620.10000000,0.06809000,110.31260900,0.00000000,USDT,1693881564753,False,False,True
3,ETHUSDT,1895165,12961820,-1,1618.96000000,0.58680000,950.00572800,0.00000000,USDT,1693881755642,False,False,True
4,ETHUSDT,1895166,12961820,-1,1618.95000000,0.36444000,590.01013800,0.00000000,USDT,1693881755642,False,False,True
5,ETHUSDT,1895167,12961820,-1,1618.90000000,0.04876000,78.93756400,0.00000000,USDT,1693881755642,False,False,True


In [61]:
# Obtener el precio actual del par de trading
precio_actual = float(cliente.ticker_price(symbol=par_de_trading)['price'])
precio_actual

1620.11

In [64]:
# Obtener información sobre el par de trading
info_par = cliente.exchange_info(symbol=par_de_trading)

# Verificar si el par de trading es válido
if info_par is None or not info_par["symbols"][0]["status"] == 'TRADING':
    print("El par de trading no es válido o no está en estado de trading.")
else:
    print("El par es valido")

El par es valido


In [ ]:
import requests

def obtener_mi_ip():
    try:
        # Hacer una solicitud HTTP a un servicio de detección de IP público
        response = requests.get('https://api64.ipify.org?format=json')

        # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            # Analizar la respuesta JSON para obtener la dirección IP
            data = response.json()
            ip = data['ip']
            return ip
        else:
            print('No se pudo obtener la dirección IP. Código de estado:', response.status_code)
            return None
    except Exception as e:
        print('Ocurrió un error al obtener la dirección IP:', str(e))
        return None

# Obtener y mostrar la dirección IP
mi_ip = obtener_mi_ip()
if mi_ip:
    print('Mi dirección IP pública es:', mi_ip)